In [8]:
# import stuff
import sys,os

import numpy as np


sys.path.append( os.environ["PYTHONSCRIPT_DIR"] )
 
import matplotlib as mpl

import PlotSettings as pS
pS.loadPlotSettings(mpl)
lineSettings = pS.generalSettings.defaultLineSettings 

from Tools.Parsers.Utilities import parseNDArray

sys.path.append( os.environ["JOBGENERATOR_DIR"])
from HPCJobConfigurator.jobGenerators.commonFunctions import CommonFunctions as cf 


import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors

#GUi shit
%matplotlib tk
mpl.get_configdir()

%load_ext autoreload
%autoreload 2

Setting matplotlib settings from file: /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/Simulations/PythonScripts/PlotSettings/matplotlibrc
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:

import glob2, re

from matplotlib.ticker import MultipleLocator
import io

plt.close("all")


def loadIniFile(file):
    # no interpolation is done!
    import configparser
    config = configparser.ConfigParser(interpolation=None)
    config.read(file)
    return config

def loadData(files,dataDimPerState=3):
    import xml.etree.ElementTree as ET
    dataPerFile = []
    for f in files:
        print("Parsing file: ", f)
        root = ET.parse(f["file"])
        
        # stack all data together
        size = len(root.findall("State"))
        dataArray = np.ndarray((size,), dtype=dataTypeRow)
        for idx,s in enumerate(root.iter("State")):
            dataArray[idx] = parseNDArray(s.text,dt=dataTypeRow)
        
        dataPerFile.append(dict(f,data=dataArray))
            
    return dataPerFile

def generateColorMap(l,name):
    jet = colorMap = plt.get_cmap(name) 
    cNorm  = colors.Normalize(vmin=0, vmax=l)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=colorMap)
    return scalarMap;
    

# Indices for data in file: ===================
dataTypeRow = [('time', float), ('stateIdx', int), ('bodyCount', int)]
dataDimPerState = 3
# =====================================

# Plot Options =======================
thisRootFolder = "../"
simulationJobsFolder =  os.path.join(thisRootFolder,"../SimulationJobs")

corrExperimentNr = 5;
figureSize= (pS.cm2inch(8),pS.cm2inch(6))

legendProps = dict(prop=dict(size=6),borderpad=0.2,labelspacing=0.1)
studyCMapName = "spectral"
xAxisLabelMinorSpacing = 0.1
xAxisLabelMajorSpacing = 0.5

studyJobIniFile = os.path.join( simulationJobsFolder, "StudyConfig.ini")

class LabelFormatter:
    def __init__(self,parameterStudyConfig):
        self.muList = cf.jsonParse(parameterStudyConfig["muList"])
    def __call__(self,studyNr, param=True):
        if param:
            return r"P-%i, $\mu=%.1f$" % (studyNr,self.muList[studyNr])
        else:
            return r"P-%i" % (studyNr)


experimentSettingsFile = os.path.join( thisRootFolder, "data/ExperimentSettings.json" )
expPlotOpts = dict(ls="-",color='black', lw=lineSettings["thick"])

def formatAxAndFig(fig,ax):
    ax.get_legend().get_frame().set_linewidth(lineSettings["thin"])
    for axis in ['top','bottom','left','right']:
          ax.spines[axis].set_linewidth(lineSettings["semi-thick"])
    
def makeStudyPlotOptions(file,idx, color):
    cc = 0.6
    c = [cc]*3
    file["plotOpts"] =dict(
            lines = dict(ls="-",color=color, lw=lineSettings["thin"]),
            linesduennG = dict(ls="-",color=c,lw=lineSettings["extra-thin"]),
            linesduenn = dict(ls="-",color=color, lw=lineSettings["thin"]),
            points = dict(marker='o',ms=2,markerfacecolor=color, markeredgewidth=0,markeredgecolor=None)
        )
# =====================================


# Glob all files
filePaths = sorted(glob2.glob("*.xml"))

files = []
for f in filePaths:
    m=re.match(".*P-(\d*)-",f);
    if(m):
        nr = int(m.group(1))
        files.append({"file":f,"studyNr":nr})

files.sort(key=lambda x: x["studyNr"])


# Load ExperimentSettingsFile
expSet = cf.jsonLoad(experimentSettingsFile)
timeEntryExperiment = expSet["experiments"]["%i"%corrExperimentNr]["deltaTimeStartTillEntry"]
timeEntryTillFinal = expSet["experiments"]["%i"%corrExperimentNr]["deltaTimeEntryTillFinal"]

# Load SimulationJob config ini
studyConfig = loadIniFile(studyJobIniFile)["ParameterStudy"]
labelFormatter = LabelFormatter(studyConfig)

# Load Data
files = loadData(files,dataDimPerState)

colorMap = generateColorMap(len(files),studyCMapName).to_rgba
#Colormap
for fIdx,f in enumerate(files):
    makeStudyPlotOptions(f,fIdx,colorMap(fIdx))
    
    
# Plot Entry Count (count of bodies entering the lower slope) ====================
fig = plt.figure(figsize=figureSize)
ax = fig.add_subplot(111)
files.reverse()
for i,f in enumerate(files):
    d = f["data"]
    ax.axvline(x=d["time"][-1], ymin=0.00, ymax=0.05, **f["plotOpts"]["linesduenn"])
    ax.plot([d["time"][-1]]*2, [0.0,1], **f["plotOpts"]["linesduennG"])
    ax.plot(d["time"][-1], 1 , **f["plotOpts"]["points"] )
for i,f in enumerate(files):
    d = f["data"]
    ax.plot(d["time"], d["bodyCount"] / d["bodyCount"][-1],  label=labelFormatter(f["studyNr"],False), **f["plotOpts"]["lines"])

    
ax.legend(loc=2,**legendProps)   
ax = plt.gca()
ax.set_ylabel(r"body count $\frac{n_{b,L}}{n_{b}}$")
ax.set_xlabel(r"time $[s]$")
# ax.set_title("Simulation Study with 3 Trees")
minorLocatorX = MultipleLocator(xAxisLabelMinorSpacing) 
ax.xaxis.set_minor_locator(minorLocatorX)
minorLocatorY = MultipleLocator(0.1) 
ax.yaxis.set_minor_locator(minorLocatorY)

majorLocatorX = MultipleLocator(0.5) 
ax.xaxis.set_major_locator(MultipleLocator(xAxisLabelMajorSpacing))
majorLocatorY = MultipleLocator(1) 
ax.yaxis.set_major_locator(majorLocatorY)

ax.grid(which='minor', alpha=0.15, linestyle=':')
ax.grid(which='major', alpha=0.4, linestyle=':') 

ax.margins(0.05, 0.05)

formatAxAndFig(fig,ax)
fig.tight_layout();
plt.subplots_adjust(left=0.17, bottom=0.2, right=0.99, top=0.91, wspace=0, hspace=0)
plt.savefig("bodyCountPlot.pdf")

# =======================================================================================


# Plot flow through (derivative of body count)
fig = plt.figure(figsize=figureSize)
ax = fig.add_subplot(111)
ax.ticklabel_format(style='sci',scilimits=(0,0),axis='y')

for i,f in enumerate(files):
    d = f["data"]
    diff = np.diff( d["bodyCount"] / d["bodyCount"][-1]) # normed flow (of total body count)
    diff = np.hstack([diff,0])
    plt.plot(d["time"][:len(diff)], diff, label=labelFormatter(f["studyNr"],True), **f["plotOpts"]["lines"])
    ax.plot(d["time"][len(diff)-1], 0 , **f["plotOpts"]["points"] )


# Mark experiment
ax.axvline(x=timeEntryExperiment, ymin=0.00, ymax=0.03, **expPlotOpts)
ax.axvline(x=timeEntryExperiment+timeEntryTillFinal, ymin=0.00, ymax=0.03, **expPlotOpts )

ax.legend(loc=1,**legendProps)
ax.get_legend().get_frame().set_linewidth(lineSettings["thin"])

ax.set_ylabel(r"body flow $\frac{\Delta n_{b,L}}{\Delta t\cdot n_{b}}$ [1/s]")
ax.set_xlabel(r"time $[s]$")
# ax.set_title("Simulation Study with 3 Trees")
ax.margins(0.08, 0.05)

ax.xaxis.set_minor_locator(minorLocatorX)
ax.xaxis.set_major_locator(majorLocatorX)
ax.yaxis.set_minor_locator(MultipleLocator(0.001))

ax.grid(which='minor', alpha=0.15, linestyle=':')
formatAxAndFig(fig,ax)
fig.tight_layout();
plt.subplots_adjust(left=0.17, bottom=0.2, right=0.99, top=0.91, wspace=0, hspace=0)
plt.savefig("bodyCountFlowPlot.pdf")

Parsing file:  {'file': 'SimState-P-0-FindStart.xml', 'studyNr': 0}
Parsing file:  {'file': 'SimState-P-1-FindStart.xml', 'studyNr': 1}
Parsing file:  {'file': 'SimState-P-2-FindStart.xml', 'studyNr': 2}
Parsing file:  {'file': 'SimState-P-3-FindStart.xml', 'studyNr': 3}
Parsing file:  {'file': 'SimState-P-4-FindStart.xml', 'studyNr': 4}
Parsing file:  {'file': 'SimState-P-5-FindStart.xml', 'studyNr': 5}
Parsing file:  {'file': 'SimState-P-6-FindStart.xml', 'studyNr': 6}
Parsing file:  {'file': 'SimState-P-7-FindStart.xml', 'studyNr': 7}
Parsing file:  {'file': 'SimState-P-8-FindStart.xml', 'studyNr': 8}
Parsing file:  {'file': 'SimState-P-9-FindStart.xml', 'studyNr': 9}
Parsing file:  {'file': 'SimState-P-10-FindStart.xml', 'studyNr': 10}
Parsing file:  {'file': 'SimState-P-11-FindStart.xml', 'studyNr': 11}
Parsing file:  {'file': 'SimState-P-12-FindStart.xml', 'studyNr': 12}
Parsing file:  {'file': 'SimState-P-13-FindStart.xml', 'studyNr': 13}
Parsing file:  {'file': 'SimState-P-14-F

In [ ]:

entryTimeSettings = {}

# Determine all enterTimes
bodyThresh = 150 # bodies

entryTimeSettings["general"] = dict(bodyThreshold=bodyThresh)
studies = entryTimeSettings["studies"] = {}

files.reverse()
for f in files:
    d= f["data"]
    idx = np.argmax(d["bodyCount"] > 150)
    idx = idx -1 if idx >=0 else 0
    studies[f["studyNr"]] = dict( 
            entryTime =  float(d["time"][idx]),
            entryStateIdx = int(d["stateIdx"][idx]),
            entryBodyCount = int(d["bodyCount"][idx]),
            finalTime = float(d["time"][-1]),
            finalStateIdx = int(d["stateIdx"][-1]),
            finalBodyCount = int(d["bodyCount"][-1])
        )

f=open("StudyEntryData.json","w")
jsonDump(entryTimeSettings,f,sort_keys=True, indent=4);
f.close()